## Compute TOA reflectance from band files and export as .txt and .tif


### 1. Imports and Configuration

In [8]:
# Cell 1: Import necessary libraries
import os
import numpy as np
import pandas as pd
from datetime import datetime
import rasterio
from tqdm import tqdm


In [ ]:
# Cell 2: Set constants and parameters

# Test data and reference paths
test_dir = r"D:\test"
cloud_mask_dir = os.path.join(test_dir, "cloud-masks")              #replace with path to your cloud_mask directory 
excel_path = r"D:\NRSC_offline\nrsc\Earth_Sun_distance.xlsx"        #replace with path to your distance file

# Sensor-specific calibration constants
Lmin = {"BAND2": 0.0, "BAND3": 0.0, "BAND4": 0.0}
Lmax = {"BAND2": 52.0, "BAND3": 47.0, "BAND4": 31.5}
Esun = {"BAND2": 185.347, "BAND3": 158.262, "BAND4": 110.81}

# Optional: add extra scene for testing
extra_scene = "BH_R2F01SEP2024069380009900053SSANSTUC00GTDC"            


### 2. TOA Reflectance Calculation and TXT Saving

In [10]:
# Cell 3: Define the TOA computation function
def compute_toa_and_save_txt(
    test_dir,
    cloud_mask_dir,
    distance_excel_path,
    Lmin,
    Lmax,
    Esun,
    output_dir="toa_txt_outputs",
    bands=["BAND2", "BAND3", "BAND4"],
    extra_scene=None
):
    os.makedirs(output_dir, exist_ok=True)

    # --- Helper: Read metadata and Earth-Sun distance ---
    def get_solar_geometry(meta_path, excel_path):
        with open(meta_path, 'r') as f:
            for line in f:
                if 'DateOfPass' in line:
                    date_of_pass = line.split('=')[1].strip()
                elif 'SunElevationAtCenter' in line:
                    sun_elev = float(line.split('=')[1].strip())

        doy = datetime.strptime(date_of_pass, '%d-%b-%Y').timetuple().tm_yday
        df = pd.read_excel(excel_path)
        earth_sun_dist = float(df[df['DOY'] == doy].iloc[0]['d'])
        return sun_elev, earth_sun_dist

    # --- Helper: Convert DN to TOA ---
    def dn_to_toa(dn, band_name, sun_elev, d):
        radiance = Lmin[band_name] + (dn / 1023.0) * (Lmax[band_name] - Lmin[band_name])
        theta_s_rad = np.deg2rad(90 - sun_elev)
        toa = (np.pi * radiance * d ** 2) / (Esun[band_name] * np.cos(theta_s_rad))
        return np.clip(toa, 0, 1)

    # --- Build scene list from cloud masks ---
    scene_ids = []

    # Use cloud_mask_dir only if it exists
    if cloud_mask_dir and os.path.isdir(cloud_mask_dir):
        scene_ids = [f.replace("cloud_", "").replace(".tif", "") for f in os.listdir(cloud_mask_dir)
                     if f.startswith("cloud_") and f.endswith(".tif")]

    # Add extra_scene explicitly
    if extra_scene:
        scene_ids.append(extra_scene)

    # Deduplicate
    scene_ids = list(set(scene_ids))


    # --- Process each scene ---
    for scene_id in scene_ids:
        scene_path = os.path.join(test_dir, scene_id)
        if not os.path.isdir(scene_path):
            print(f"[SKIP] Folder not found for {scene_id}")
            continue

        meta_path = os.path.join(scene_path, "BAND_META.txt")
        if not os.path.exists(meta_path):
            print(f"[SKIP] Metadata missing for {scene_id}")
            continue

        try:
            sun_elev, earth_sun_dist = get_solar_geometry(meta_path, distance_excel_path)
            for band in bands:
                band_path = os.path.join(scene_path, f"{band}.tif")
                if not os.path.exists(band_path):
                    print(f"[WARN] Missing {band}.tif for {scene_id}")
                    continue

                with rasterio.open(band_path) as src:
                    dn = src.read(1).astype(np.float32)
                    toa = dn_to_toa(dn, band, sun_elev, earth_sun_dist)

                # Save TOA array to .txt
                txt_path = os.path.join(output_dir, f"{scene_id}_{band}_toa.txt")
                np.savetxt(txt_path, toa, fmt="%.6f")

            print(f"[DONE] Saved TOA arrays to .txt for {scene_id}")

        except Exception as e:
            print(f"[ERROR] {scene_id}: {str(e)}")


In [11]:
compute_toa_and_save_txt(
    test_dir=test_dir,
    cloud_mask_dir="empty_or_dummy_folder",  # Your cloud_mask directory
    distance_excel_path=excel_path,      
    Lmin=Lmin,
    Lmax=Lmax,
    Esun=Esun,
    output_dir=os.path.join(test_dir, "toa_txt_outputs"),
    extra_scene="BH_R2F01SEP2024069380009900053SSANSTUC00GTDC"  # Just this scene for testing
)


[DONE] Saved TOA arrays to .txt for BH_R2F01SEP2024069380009900053SSANSTUC00GTDC


### 3. Convert TXT to GeoTIFF Using Reference .tif

In [14]:
def convert_txt_to_tif_with_progress(txt_folder, tif_reference_folder, output_folder, target_scenes=None):
    """
    Converts TOA .txt files to .tif using spatial reference from original .tif files.

    Parameters:
    - txt_folder: Path to folder with *_toa.txt files.
    - tif_reference_folder: Folder containing subfolders with original .tif bands.
    - output_folder: Where to save .tif files.
    - target_scenes: Optional list of scene_ids to filter which files to convert.
    """
    os.makedirs(output_folder, exist_ok=True)

    # All txt files
    txt_files = [f for f in os.listdir(txt_folder) if f.endswith("_toa.txt")]

    # Filter for specific scenes if provided
    if target_scenes:
        txt_files = [f for f in txt_files if any(f.startswith(scene_id) for scene_id in target_scenes)]

    for fname in tqdm(txt_files, desc="Converting TOA TXT to TIF", unit="file"):
        scene_band = fname.replace("_toa.txt", "")
        scene_id, band = scene_band.rsplit("_", 1)

        txt_path = os.path.join(txt_folder, fname)
        ref_tif_path = os.path.join(tif_reference_folder, scene_id, f"{band}.tif")
        out_tif_path = os.path.join(output_folder, scene_band + "_toa.tif")

        if not os.path.exists(ref_tif_path):
            tqdm.write(f"[SKIP] Reference .tif not found for {scene_id} {band}")
            continue

        try:
            toa_array = np.loadtxt(txt_path).astype(np.float32)

            with rasterio.open(ref_tif_path) as ref:
                profile = ref.profile.copy()
                profile.update({
                    "dtype": "float32",
                    "count": 1,
                    "compress": "lzw"
                })

                with rasterio.open(out_tif_path, "w", **profile) as dst:
                    dst.write(toa_array, 1)

        except Exception as e:
            tqdm.write(f"[ERROR] {scene_id} {band}: {str(e)}")

    print("\n✅ Conversion complete!")


In [15]:
txt_folder = r"D:\test\toa_txt_outputs"
tif_reference_folder = r"D:\test"
output_folder = r"D:\test\toa_tif_outputs"

# Only convert one scene:
convert_txt_to_tif_with_progress(
    txt_folder,
    tif_reference_folder,
    output_folder,
    target_scenes=["BH_R2F01SEP2024069380009900053SSANSTUC00GTDC"]
)


Converting TOA TXT to TIF: 100%|██████████| 3/3 [02:52<00:00, 57.57s/file]


✅ Conversion complete!
